<p> Run a simulation of filling in a network based on the 7 de Septiembre neighborhood layout in Araijan </p>
<p>**(.inp and .config files already written)**</p>
<p> Below you see how to:</p>
<ul>
<li>Set boundary and initial conditions </li>
<li>Run a simulation until time T= 1200 s</li>
<li>plot time series of pressure head at different points in a single pipe</li>
<li>plot time series of pressure head at sample points in various pipes</li>
<li>plot space dependence of pressure head at a certain time</li>
<li>look at velocities</li>
<li>plot network layout</li>
</ul>

In [1]:
from __future__ import division
import sys
sys.path.append("/home/xin/pipes")  
from allthethings import PyNetwork, PyPipe_ps
from allthethings import PyBC_opt_dh
import numpy as np
import matplotlib.pyplot as plt
#%pylab inline
%pylab notebook
from writeit import *
import pickle
import time

fi = "/home/xin/pipes/indata/Xindata/SaketTrunkMainMap-2.1.inp"    
fc = "/home/xin/pipes/indata/Xindata/SaketTrunkMainMap-2.0.config" 
mtype = 1                          
n1 = PyNetwork(fi,fc,mtype)

Populating the interactive namespace from numpy and matplotlib


In [ ]:
#################   Get Elevation from Input File ###################
def getElev(fi):
    elev = []
    f = open(fi, 'r')
    Bool = False
    for line in f:
        label = 0
        line = line.split()
        if line and line[0]== "[PIPES]":
            return elev
        if Bool and line:
            elev.append(float(line[1]))
        if line and line[0] == ";ID":
            Bool = True

elev = getElev(fi)
print 'Elevtaion Got'


########################## Set IC for all pipes #################################
bench_H = 234.26  #### Initial Water Depth 

##### Given a target depth H, set IC for a certain pipe i #######
def SetPipeIC (n1, i, H, left, right):
    L = n1.Ls[i]
    N = n1.Ns[i]
    D = n1.Ds[i]
    dx = L/N
    diff = (right-left)/N
    pipe = PyPipe_ps(N,D,L,n1.M,n1.a[0])
    A = []
    A_crt = 1e-4
    Q_crt =5e-5
    for k in xrange(N):
        current = left + (k+1/2)*diff
        deltaH = H - current
        if deltaH <= pipe.HofA(A_crt, False):
            A.append(A_crt)
        else:
            A.append( pipe.AofH(deltaH, True) )
    Q = Q_crt*np.ones(N)
    n1.setIC(i, A, Q)


############ Inflpow Boundary Values (m^3/s) #########################
q0 = 0.3   #inflow boundary value (m^3/s), used to create the Q0 vector
######################################################################
M = n1.M  
T = n1.T   
Q0 = q0*np.ones(M+1)          
n1.setbVal(0,Q0)   #boundary condition for junction 0 (at the left end of pipe 0)
print 'Inflow boundary finished'


'''
The Optimal orfice here should be set as follows:
1) Fully open until pressurized (not ploted in field data)
2) Gradually decrease the opening to 0.1 & 0.05 (reach the first stage)
3) Keep there for 4 minutes
4) Gradually decrease the opening to 0.0729 & 0.0176 (or a higher number consider 15m head)
'''

M1 = int((M+1)/3)
Open1 = np.ones(M+1)
Open1[:M1+1] = [0.16 - 0.06*i/M1 for i in xrange(len(Open1[:M1+1]))]
Open1[M1+1:2*M1+2] = [0.1 for i in Open1[M1+1:2*M1+2]]
Open1[2*M1+2:] = [0.1 - (0.1-0.0729)*i/M1 for i in xrange(len(Open1[2*M1+2:]))]
n1.setbVal(1,Open1)

Open2 = np.ones(M+1)
Open2[:M1+1] = [0.07-0.02*i/M1 for i in xrange(len(Open1[:M1+1]))]
Open2[M1+1:2*M1+2] = [0.05 for i in Open2[M1+1:2*M1+2]]
Open2[2*M1+2:] = [0.05 - (0.05-0.0176)*i/M1 for i in xrange(len(Open2[2*M1+2:]))]
n1.setbVal(2,Open2)


'''
########## Get some information about the pipe pressure head when full #######################
a = n1.a[0]
for i in xrange(len(n1.Ns)):
    p1 = PyPipe_ps(n1.Ns[i], n1.Ds[i],n1.Ls[i], M, a)
    Ap = n1.Ds[i]**2/4*np.pi  # For the first pipe, get the pipe cross-section area
    g = 9.81
    p_head = p1.Eta(Ap,True)/(g*Ap)
    slot_w = g*Ap/a**2
    print "For pipe %d, when it is full, its pressure head is %.4f m, slot width is %.3e"%(i,p_head, slot_w)
print 'Pipe info got '
'''   
'''   
########## Set Initial Condition #######################################
for i in xrange(0,len(n1.Ns)):
    A00 = 1e-4*np.ones(n1.Ns[i])
    Q00 = 5e-5*np.ones(n1.Ns[i])
    n1.setIC(i,A00,Q00)
print 'IC setup finished'
''' 



Elevtaion Got
Inflow boundary finished


"   \n########## Set Initial Condition #######################################\nfor i in xrange(0,len(n1.Ns)):\n    A00 = 1e-4*np.ones(n1.Ns[i])\n    Q00 = 5e-5*np.ones(n1.Ns[i])\n    n1.setIC(i,A00,Q00)\nprint 'IC setup finished'\n"

In [ ]:
############### Set Parameters for loop ###########################
T_total = 3600
###################################################################

# i = 0 means A, i=1 means Q
# j means the grid we are currently looking at, 1<=j<=N
# n means time step 
# N means the grid number in this certain pipe
def idx_t(i,j,n,N):
    return (2*(N+2)*n+(N+2)*i+j)

# conn: np.ndarray
#       Nedgesx2 array of ints. Row i = [start node, end node] for pipe i.
pipe_number = len(n1.conn)
model_number = int(T_total/T)
dt = T/float(M)
# Store all the pressure head data before T
P_beforeT = [] 
# Store all the pressure head data after T
P_afterT = []  
# Store the IC condition for each round of simulation
# ICs = { pipe: [ [A],[Q] ] }, therefore A = ICs[pipe][0], Q = ICs[pipe][1] 
ICs = {}
for k in xrange(model_number):
    n1.setbVal(0,Q0) 
    if k != 0:
        Open1 = 0.0729*np.ones(M+1)
        n1.setbVal(1,Open1)
        Open2 = 0.0176*np.ones(M+1)
        n1.setbVal(2,Open2)
        t1 = time.time()
        for pipe in xrange(pipe_number):
            n1.setIC(pipe, ICs[pipe][0], ICs[pipe][1])
        n1.runForwardProblem(dt)
        t2 = time.time()
    else:
        t1 = time.time()
        for pipe in xrange(pipe_number):
            edge = n1.conn[pipe]
            SetPipeIC(n1, pipe, bench_H, elev[edge[0]], elev[edge[1]])
        n1.runForwardProblem(dt)
        t2 = time.time()
    print 'Piece %d of %d, running time = %ds'%(k+1, model_number, t2-t1)
    for pipe in xrange(pipe_number):
        qh = n1.qhist(pipe)
        N = n1.Ns[pipe]
        A = [qh[idx_t(0, j, M, N)] for j in xrange(1,N+1)]
        Q = [qh[idx_t(1, j, M, N)] for j in xrange(1,N+1)]
        ICs.update({pipe: [A, Q]})
    if len(P_beforeT)>0: 
        # Cut the last number since the begining and the end are overlaped
        P_beforeT = np.concatenate((P_beforeT[:-1], n1.pressureTimeSeries(2,n1.Ns[2]-1)), axis = 0)  #One grid before the last one 
        P_afterT = np.concatenate((P_afterT[:-1], n1.pressureTimeSeries(4,0)), axis = 0)
    else:
        P_beforeT = n1.pressureTimeSeries(2,n1.Ns[2]-1)
        P_afterT = n1.pressureTimeSeries(4,0)
    n1.reset()
        

In [ ]:
###########  Subplots: Junction Pressure, Qout, Orifice, In/Out Pressure ####################
import matplotlib.dates as mdates
################## Define Start Time String and End Time String Based on Simulation ######################
start_time = '8/23/2016 05:03:05'
end_time = '8/23/2016 05:18:05'
frac = 0.2
Mi = 200
##################################################################################################

delta_t = n1.T/n1.M * 1000  # in millisecond，because delta in the following function is in 微秒
M_scale = int((n1.M+1)*frac)
print 'Total Time we are looking at:', M_scale/n1.M*n1.T,'s'

# 'start' is a string in H:M:S format, so is 'end'. 'delta' is an int with unit: ms
def date_range(start, end, delta):    
    from datetime import datetime, timedelta
    FMT = "%m/%d/%Y %H:%M:%S"
    start = datetime.strptime(start,FMT)
    end = datetime.strptime(end,FMT) 
    diff = timedelta(milliseconds = delta)
    times = []
    time = start
    while time <= end:
        times.append(time)  #.time() method to extract hour, minutes, seconds and microseconds from datetime
        time = time + diff
    return times

#----------------------------------------------------------------------------
# Define a function to get time average to suppress oscillation

# t is the initial x value that corresponds to P
# P is the pressure that should be averaged over time
# Mi is how many time steps you want to average 
def t_average_time(t, P, Mi):
    n = int(len(P[:-1])/Mi)    # Totally M steps in P{:-1], n=M/Mi, n should be an integer
    x = []          
    P_new = []
    for i in xrange(n):
        P_new.append(np.average(P[i*Mi:(i+1)*Mi-1]))
        x.append(t[int((i+0.5)*Mi)])
    return x, P_new
#--------------------------------------------------------------
# Define a function to put the legend at the right center out of the plot, ax = plt.subplot or plt.plot
def right_legend(ax):
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))


################ Common X axies for all subplots ##############################
t = date_range(start_time, end_time,delta_t)
plt.figure(figsize= (10,20))   
plt.rcParams['xtick.major.pad']='8'   # Distance between edges and xticks
plt.xlabel('Time')
plt.show()
######################################################################
ax1 = plt.subplot(511)             # P-t

print "Now the delta t is", Mi*n1.T/n1.M, 's'
t1n, P_beforeTn = t_average_time(t, P_beforeT, Mi)
t2n, P_afterTn = t_average_time(t, P_afterT, Mi)
ax1.plot(t1n[:int(M_scale/Mi)], P_beforeTn[:int(M_scale/Mi)], 'r', label ='Before T')   ## Averaged, so rescale
ax1.plot(t2n[:int(M_scale/Mi)], P_afterTn[:int(M_scale/Mi)],'b', label ='After T')
#legend(bbox_to_anchor=(0.9, 1), loc=7, borderaxespad=0.)
#legend().draggable()
#legend(loc='best')
myFmt = mdates.DateFormatter('%H:%M:%S')
ax1.xaxis.set_major_formatter(myFmt)
ax1.set_ylabel('Pressure Head (m)')
ax1.set_title('Before/After Junction H-t')
right_legend(ax1)
plt.setp(ax1.get_xticklabels(), visible=False)  # Make the x ticks invisible

#######################################################################
ax2 = plt.subplot(512, sharex=ax1)             # Q-t
Out = [5,6] # Out1, Out2
# Calculate Q1, Q2 distribution
#----------------------------------------------------------------------------
# get pipe (A,Q)data
qh1 = n1.qhist(Out[0])
qh2 = n1.qhist(Out[1])
N1 = n1.Ns[Out[0]]
N2 = n1.Ns[Out[1]]
Qext1 = [qh1[idx_t(1,N1+1,it,N1)] for it in xrange(n1.M+1)]
Qext2 = [qh2[idx_t(1,N2+1,it,N2)] for it in xrange(n1.M+1)]
print "Q1 finally =",Qext1[-1], "Percentage =", Qext1[-1]/q0
print "Q2 finally =",Qext2[-1],"Percentage =", Qext2[-1]/q0
t_qext1n, Qext1n = t_average_time(t, Qext1, Mi)
t_qext2n, Qext2n = t_average_time(t, Qext2, Mi)
ax2.plot(t_qext1n[:int(M_scale/Mi)], Qext1n[:int(M_scale/Mi)], label = 'Main')
ax2.plot(t_qext2n[:int(M_scale/Mi)], Qext2n[:int(M_scale/Mi)], label = 'Branch')
#legend(bbox_to_anchor=(0.9, 1), loc=7, borderaxespad=0.)
#legend(loc='best')
ax2.set_ylabel("flux(m^3/s)")
ax2.set_title('Q-t')
right_legend(ax2)
plt.setp(ax2.get_xticklabels(), visible=False)  # Make the x ticks invisible

#######################################################################
ax3 = plt.subplot(513, sharex=ax1)             # Orifice -t 
Mx = arange(M+1)*T/M/60
ax3.plot(t[:M_scale], Open1[:M_scale], label = 'Orifice Main')
ax3.plot(t[:M_scale], Open2[:M_scale], label = 'Orifice Branch')
ax3.set_ylabel('orifice(m)')
ax3.set_title("Orifice-t")
right_legend(ax3)
plt.setp(ax3.get_xticklabels(), visible=False)  # Make the x ticks invisible

#######################################################################
ax4 = plt.subplot(514, sharex=ax1)          # Inlet P- t
In = [0]
# Calculate Q1, Q2 distribution
#----------------------------------------------------------------------------
# get pipe (A,Q)data
P_in = n1.pressureTimeSeries(In[0],0)
t_inn, P_inn = t_average_time(t, P_in, Mi)
ax4.plot(t_inn[:int(M_scale/Mi)], P_inn[:int(M_scale/Mi)], label = 'Inlet')
ax4.set_ylabel("Pressure Head(m)")
ax4.set_title('Inlet H-t')
plt.setp(ax4.get_xticklabels(), visible=False)
right_legend(ax4)


#######################################################################
ax5 = plt.subplot(515, sharex=ax1)          # Outlet P - t
Out = [5,6] # Out1, Out2
# Calculate Q1, Q2 distribution
#----------------------------------------------------------------------------
# get pipe (A,Q)data
P_out1 = n1.pressureTimeSeries(Out[0],n1.Ns[Out[0]]-1)
P_out2 = n1.pressureTimeSeries(Out[1],n1.Ns[Out[1]]-1)
t_out1n, P_out1n = t_average_time(t, P_out1, Mi)
t_out2n, P_out2n = t_average_time(t, P_out2, Mi)
ax5.plot(t_out1n[:int(M_scale/Mi)], P_out1n[:int(M_scale/Mi)], label = 'Main Outlet')
ax5.plot(t_out2n[:int(M_scale/Mi)], P_out2n[:int(M_scale/Mi)], label = 'Branch Outlet')
ax5.set_ylabel("Pressure Head(m)")
ax5.set_title('Outlet H-t')
plt.xticks(rotation=30)   #  Make room for x ticks
right_legend(ax5)





In [ ]:
######  Only For Check, do not need to run #############
'''
A0 = np.zeros(M+1)
n1.setbVal(0,A0)
n1.setbVal(1,A0)
n1.setbVal(2,A0)
'''
M = n1.M
a = n1.a[0]
p0 = PyPipe_ps(n1.Ns[0], n1.Ds[0],n1.Ls[0], M, a)
p6 = PyPipe_ps(n1.Ns[6], n1.Ds[6],n1.Ls[6], M, a)
A0 = p0.AofH(0.8, True)
A6 = p6.AofH(0.625, True)
print p0.HofA(0.02, False)
print p6.HofA(0.00128, False)
#print p0.pbar(A0, False), p6.pbar(A6, False) , p0.HofA(0.290475, True), p0.AofH(0.0729*0.83, True)



In [ ]:
########### Plot model layout ##############################
"""
(xs,ys,conns,ls) = getBasicConnectivity(fi)
Np= shape(conns) [0]
%pylab notebook
plotNetworkLayout (xs, ys, conns, ls, Np) 
"""
########### Print initial data about the model ############# (Not necessary, but can check)
#n1.showLayout()
#n1.showCurrentData()

In [ ]:
#----------------------------------------------------------------------------
# Calculate Pressure Head Difference 
#----------------------------------------------------------------------------
In = [2]    # In pipe
Out = [3,6] # Out1, Out2
#######################################################################

P0 = n1.pressureTimeSeries(In[0],n1.Ns[In[0]]-1) 
P1 = n1.pressureTimeSeries(Out[0],0)
P2 = n1.pressureTimeSeries(Out[1],0)
t = linspace(0,n1.T, n1.M+1)
# Pressure Head difference dP1 = P0 - P1
dP1 = [P1[i]-P0[i] for i in xrange(len(P0))]
dP2 = [P2[i]-P0[i] for i in xrange(len(P0))]
print "dP1 finally =",dP1[-1]
print "dP2 finally =",dP2[-1]
fig = plt.figure(figsize= (10,5))
plot(t,dP1, label = 'P1-P0')
plot(t,dP2, label = 'P2-P0')
legend()
xlabel("time")
ylabel("Pressue Head (m)")
plt.ylim(-2.5,2.5)

#----------------------------------------------------------------------------
# Calculate Q1, Q2 distribution
#----------------------------------------------------------------------------
# get pipe (A,Q)data
qh0 = n1.qhist(In[0])
qh1 = n1.qhist(Out[0])
qh2 = n1.qhist(Out[1])
N = n1.Ns[1]
def idx_t(i,j,n,N):
    return (2*(N+2)*n+(N+2)*i+j)
N1 = n1.Ns[Out[0]]
N2 = n1.Ns[Out[1]]
Qext1 = [qh1[idx_t(1,N1,it,N1)] for it in xrange(n1.M+1)]
Qext2 = [qh2[idx_t(1,N2,it,N2)] for it in xrange(n1.M+1)]
print "Q1 finally =",Qext1[-1], "Percentage =", Qext1[-1]/q0
print "Q2 finally =",Qext2[-1],"Percentage =", Qext2[-1]/q0
fig = plt.figure(figsize= (10,5))
plot(t,Qext1, label = 'Pipe %d outflow'%Out[0])
plot(t,Qext2, label = 'Pipe %d outflow'%Out[1])
legend()
xlabel("time")
ylabel("flux(m^3/s)")
plt.ylim(0, q0)


Px = n1.pressureTimeSeries(5,n1.Ns[5]-1) 
print "Pressure Head in Pipe 0 is: ",Px[-1], "m"


In [ ]:
# Display the inflow and outflow for each pipe (m^3)  (ALl PIPES IN MODEL)


# Get Q of two ends of each pipe
Q = []
Mstep=1
#pipe_interest=[0]
pipe_interest=range(0,3)   
'''Here I redefine the pipe_interest vector'''

for i in range(len(pipe_interest)):
    Q_pipefix = [] 
    Q_start=[]
    Q_end=[]
    for m in xrange(0,n1.M,Mstep) :
        j = pipe_interest[i]
        N = n1.Ns[j]
        qh = n1.qhist(j)
        #Q_start.append(qh[idx_t(1,1,m,N)])   # Q1 (since qhist include ghost cell, so 0 and N+1 is the value of ghost cell)
        #Q_end.append(qh[idx_t(1,N,m,N)])     # QN
        Q_start.append(qh[idx_t(0,0,m,N)])   # Q1 (since qhist include ghost cell, so 0 and N+1 is the value of ghost cell)
        Q_end.append(qh[idx_t(1,N+1,m,N)])     # QN
    Q_pipefix.append(Q_start)
    Q_pipefix.append(Q_end)
    Q.append(Q_pipefix)
xt = np.linspace(0, n1.M*dt, n1.M/Mstep)
endorstart=['start','end']
for i in range(len(pipe_interest)):
    plt.figure(figsize= (10,4))                # same pipe on same figure
    for j in range(0,2):  
        #figsize = (15,5)                        # plot on same figure           
        #plt.figure(figsize= (15,5))             # plot on different figures
        plot(xt,Q[i][j], lw = 1,label = 'pipe %d, %s'%(pipe_interest[i],endorstart[j]))
        legend(bbox_to_anchor=(0.7, 0.3), loc=2, borderaxespad=0.)   #legend will be placed out of box and cannot be shown
        #legend()
        xlabel('time (s)')
        ylabel('flux (m^3/s)')
        title('Flux in pipe %d'%pipe_interest[i])
        plt.ylim(0,0.3)

# calculate inflow and outflow, i means pipe number, j means start or end, time means time in vector  
delta_t=Mstep*dt
Q_total=[]
for i in range(len(pipe_interest)):
    Q_total_pipefix=[]
    for j in range(0,2):
        Q_sum=0
        for time in range(len(Q[i][j])):
            Q_sum += Q[i][j][time]*delta_t
        Q_total_pipefix.append(Q_sum)
        print 'Pipe %d during %d s have total flow: %.3f m^3 (%s node)'%(pipe_interest[i],n1.T,Q_sum,endorstart[j])
    V_dif = Q_total_pipefix[0]- Q_total_pipefix[1]
    print V_dif
    Q_total.append(Q_total_pipefix)


# Display mass balance for each junction

#******************************************************************************************
#The following three list must corresponds to each other 
JunNode=[1]
Pipejun_in=[[0]]
Pipejun_out=[[1,2]]
#******************************************************************************************


assert (len(Pipejun_in)==len(Pipejun_out)),"Different dimensions of Pipejun_in and Pipejun_out!"

# Create x axis
xt = np.linspace(0, n1.M*dt, n1.M/Mstep)

# Q[i][j][k]   i means pipe number, j means start or end, k means time in vector 
for i in xrange(len(Pipejun_in)):
    Qjun_dif = []
    print len(Q[0])
    for k in xrange(len(Q[0][1])):
        Qinflow = 0
        Qoutflow = 0
        for j1 in Pipejun_in[i]:
            Qinflow += Q[j1][1][k]
        for j2 in Pipejun_out[i]:
            Qoutflow += Q[j2][0][k]
        Qjun_dif.append(Qinflow-Qoutflow)
    plt.figure(figsize= (10,4)) 
    plot(xt,Qjun_dif)   
    xlabel('t(s)')
    ylabel('Flux difference (m^3/s)')
    title('Junction %d Mass Conservation (Qin-Qout)'%JunNode[i])


In [ ]:
##### Continuous Animation, Fiexed Opening ########

from matplotlib import animation
#*********************************************************************************
pipe_interest=[0,1,2,3,4,5]
outflow_pipe =[5,6]
orifice = [0.0729, 0.0176]
t_delta=0.1              # unit: s
t_start=0               # unit:s
t_end=300               # unit: s
#*********************************************************************************

# animation already imported above
plt.ion()
plt.show()

# create x axis
x_interest=[]
pipe_length=0
for j in pipe_interest: 
    x = np.linspace(pipe_length,pipe_length+n1.Ls[j],n1.Ns[j])
    x_interest.append(x)
    pipe_length=pipe_length+n1.Ls[j]
x_interest_combine = np.concatenate(x_interest,axis =0)


#create initial figure
fig = plt.figure(figsize= (10,5))
plt.xlim(0,x_interest[-1][-1]+1)
plt.ylim(min(elev), 240)
lines = [plt.plot([], [])[0] for i in range(len(pipe_interest))] # number of lines plot on the figure
plt.xlabel('x (m)')        
plt.ylabel('Pressure Head (m)')
plt.title('Pressure Head in pipe %s'%str(pipe_interest))

pipe_length = 0
## Record the sequence of outflow pipe
out = 0
for pipe in pipe_interest:
    # create x axis
    x = np.linspace(pipe_length,pipe_length+n1.Ls[pipe],n1.Ns[pipe])
    
    # create elevation line
    edge = n1.conn[pipe]
    left = elev[edge[0]]
    right = elev[edge[1]]
    diff = right - left
    He = [left + diff/(len(x)-1)*i  for i in xrange(len(x))]
    plt.plot(x,  He, 'b')
    
    # create pipe full line
    p0 = PyPipe_ps(n1.Ns[pipe], n1.Ds[pipe],n1.Ls[pipe], M, n1.a[0])
    A0 = n1.Ds[pipe]*n1.Ds[pipe]/4*3.14
    H_full = [p0.Eta(A0,True)/9.81/A0]*len(x) 
    H_full = (np.add(H_full, He)).tolist()
    plt.plot(x, H_full,'--',label= 'pipe %d full'%(pipe))
    
    # If pipe is also outflow pipe
    if pipe in outflow_pipe:
        A_open = p0.AofH(orifice[out], False)
        H_open= [p0.pbar(A_open,False)]*len(x)
        print Orifice[out],A_open, H_open[0] 
        H_open =  (np.add(H_open, He)).tolist()
        plt.plot(x,H_open,'r--',label = "pipe %d orifice"%(pipe))
        out += 1
    pipe_length += n1.Ls[pipe]
    
#legend(bbox_to_anchor=(0.9, 1), loc=2, borderaxespad=0.)
# initialization function: plot the background of each frame


def init():
    for line in lines:
        line.set_data([], [])
    return lines

dt = n1.T/n1.M
Mi_draw=int(t_delta/dt)   # difne how many steps to skip
print Mi_draw, dt
M_start=int(t_start/dt)
M_total=int(t_end/dt)
H_interest=[]


# H_interested_fixed = [ [H1], [H2],... [Hn] ], it stores all pipe's pressure info @ certain time
# H_interest = [ [PressureInfo @ time 1], [Pressure info @ time 2],..,[ Pressure Info @ time m]   ]
def animate(index):
    H_interest_tfixed = []
    for i,line in enumerate(lines):  
        ## get pipe j
        j = pipe_interest[i]
        #this returns H as a function of x in pipe j at time step m
        Hx = n1.pressureSpaceSeries(j,M_start+index*Mi_draw) 
        # Add elevation to the pressure head
        # nd.array can add list to get a new array like matrix sum
        edge = n1.conn[j]
        left = elev[edge[0]]
        right = elev[edge[1]]
        diff = right - left
        Nj = n1.Ns[j]
        He = [left + diff/Nj*(i+0.5)  for i in xrange(Nj)]
        Hx = Hx + He
        H_interest_tfixed.append(Hx)
    H_interest.append(H_interest_tfixed)
    H_interest_combine = np.concatenate(H_interest[index],axis =0)
    line.set_data(x_interest_combine,H_interest_combine)
    plt.xlabel('x (m), t=%ds'%((M_start+index*Mi_draw)*dt))
                      
    return lines         
    #legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    

step = int((M_total-M_start)/Mi_draw)
anim = animation.FuncAnimation(fig, animate, init_func=init,frames=step, interval=3, blit=False)
plt.show()
"""Save Anna Result"""
#anim.save('/home/xin/pipes/examples/output_data/MyTtest/AnnaCode %s %.0f-%.0f s.mp4'%(str(pipe_interest),t_start,t_end))
"""Save My Result"""
#anim.save('/home/xin/pipes/examples/output_data/MyTtest/Karney_Improvement/Anna+B_1.11_6_open0.8/D2=%s/%s %.0f-%.0f s.mp4'%(str(n1.Ds[2]),str(pipe_interest),t_start,t_end))
#anim.save('/home/xin/pipes/examples/output_data/MyTtest/FieldCalibrate/Oct29_Pressure/%s %.0f-%.0f s.mp4'%(str(pipe_interest),t_start,t_end))

In [ ]:
#### Overlap Anmiation ############

#*********************************************************************************
pipe_interest=[5,6]
outflow_pipe =[5,6]
Orifice = [0.0729, 0.0176]
t_delta=0.1              # unit: s
t_start=0              # unit:s
t_end= 300               # unit: s
#*********************************************************************************

# create x axis
x_interest=[]
pipe_length=0
for j in pipe_interest: 
    x = np.linspace(0,n1.Ls[j],n1.Ns[j])
    x_interest.append(x)

#create initial figure
from matplotlib import animation
fig = plt.figure(figsize= (10,5))
plt.xlim(0,x_interest[-1][-1]+1)
plt.ylim(min(elev), 240)
lines = [plt.plot([], [],label='pipe {}'.format(pipe_interest[i]))[0] for i in range(len(pipe_interest))] # number of lines plot on the figure
plt.xlabel('x (m)')        
plt.ylabel('Pressure Head (m)')
plt.title('Pressure Head in pipe %s'%str(pipe_interest)) 

## Record the sequence of outflow pipe
out = 0

for pipe in pipe_interest:
    # create x axis
    x = np.linspace(0, n1.Ls[pipe], n1.Ns[pipe])
    
    # create elevation line
    edge = n1.conn[pipe]
    left = elev[edge[0]]
    right = elev[edge[1]]
    diff = right - left
    He = [left + diff/(len(x)-1)*i  for i in xrange(len(x))]
    plt.plot(x,  He, 'black')
    
    # create pipe full line
    p0 = PyPipe_ps(n1.Ns[pipe], n1.Ds[pipe],n1.Ls[pipe], M, n1.a[0])
    A0 = n1.Ds[pipe]*n1.Ds[pipe]/4*3.14
    H_full = [p0.Eta(A0,True)/9.81/A0]*len(x) 
    H_full = (np.add(H_full, He)).tolist()
    plt.plot(x, H_full,'b--',label= 'pipe %d full'%(pipe))
    
     # If pipe is also outflow pipe
    if pipe in outflow_pipe:
        A_open = p0.AofH(orifice[out], False)
        H_open= [p0.pbar(A_open,False)]*len(x)
        print Orifice[out],A_open, H_open[0] 
        H_open =  (np.add(H_open, He)).tolist()
        plt.plot(x,H_open,'g--',label = "pipe %d orifice"%(pipe))
        out += 1
    pipe_length += n1.Ls[pipe]
    
legend(bbox_to_anchor=(0.7, 1), loc=2, borderaxespad=0.)
# initialization function: plot the background of each frame


def init():
    for line in lines:
        line.set_data([], [])
    return lines

dt = n1.T/n1.M
Mi_draw=int(t_delta/dt)   # difne how many steps to skip
M_start=int(t_start/dt)
M_total=int(t_end/dt)

abc= n1.pressureSpaceSeries(2,1000)
print type(abc),len(abc)

def animate(index): 
    for i,line in enumerate(lines):             
        j=pipe_interest[i]
        #this returns H as a function of x in pipe j at time step m
        Hx = n1.pressureSpaceSeries(j,M_start+index*Mi_draw)  
        
        edge = n1.conn[j]
        left = elev[edge[0]]
        right = elev[edge[1]]
        diff = right - left
        Nj = n1.Ns[j]
        He = np.array([left + diff/Nj*(n+0.5)  for n in xrange(Nj)])
     
        Hx = Hx + He
        
        line.set_data(x_interest[i],Hx)
        plt.xlabel('x (m), t=%ds'%((M_start+index*Mi_draw)*dt))
        #line.set_label('pipe %d t=%.2f s'%(j,(dt*(M_start+index*Mi_draw))))
                      
    return lines         
    #legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    

step = int((M_total-M_start)/Mi_draw)
anim = animation.FuncAnimation(fig, animate, init_func=init,frames=step, interval=3, blit=True)
plt.legend(bbox_to_anchor=(0.85, 1), loc=2, borderaxespad=0.)
plt.show()
"""Save My Result"""
#anim.save('/home/xin/pipes/examples/output_data/MyTtest/Karney_Improvement/Anna+B_1.11_6_open0.8/D2=0.2/%s %.0f-%.0f s.mp4'%(str(pipe_interest),t_start,t_end))
#anim.save('/home/xin/pipes/examples/output_data/MyTtest/FieldCalibrate/Oct29_Pressure/%s overlap %.0f-%.0f s.mp4'%(str(pipe_interest),t_start,t_end))



#anim = animation.FuncAnimation(fig, animate, init_func=init,frames=3, interval=20, blit=True)
#anim.save("basic_animation.mp4")
#plt.show()
    # time.sleep(0.01)
    # bbox_to_anchor: the bbox that the legend will be anchored, 1.05 means 1.05 times of figure length
    # borderaxespad: the pad between the axes and legend border
# Display the inflow and outflow for each pipe (m^3)  (ALl PIPES IN MODEL)




In [ ]:
##### Subplot, Do not work now ######
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.animation as animation


# This example uses subclassing, but there is no reason that the proper
# function couldn't be set up and then use FuncAnimation. The code is long, but
# not really complex. The length is due solely to the fact that there are a
# total of 9 lines that need to be changed for the animation as well as 3
# subplots that need initial set up.
class SubplotAnimation(animation.TimedAnimation):
    def __init__(self):
        #*********************************************************************************
        pipe_interest1=[0,1,2,3,4,5]
        pipe_interest2=[0,1,2,6]
        outflow_pipe =[5,6]
        Orifice = [0.0729, 0.0176]
       
        #*********************************************************************************

        x_interest=[]
        pipe_length=0
        for j in pipe_interest1: 
            x = np.linspace(pipe_length,pipe_length+n1.Ls[j],n1.Ns[j])
            x_interest.append(x)
            pipe_length=pipe_length+n1.Ls[j]
        x_interest_combine1 = np.concatenate(x_interest,axis =0)
        
    
        x_interest=[]
        pipe_length=0
        for j in pipe_interest2: 
            x = np.linspace(pipe_length,pipe_length+n1.Ls[j],n1.Ns[j])
            x_interest.append(x)
            pipe_length=pipe_length+n1.Ls[j]
        x_interest_combine2 = np.concatenate(x_interest,axis =0)
        
        fig = plt.figure()
        ax1 = fig.add_subplot(1, 2, 1)
        ax2 = fig.add_subplot(2, 2, 2)
        
        self.x1 = x_interest_combine1
        self.x2 = x_interest_combine2
        self.p1 = 0
        self.p2 = 0

        #ax1.set_xlabel('x')
        ax1.set_ylabel('H(m)')
        self.line1 = Line2D([], [], color='black')
        ax1.add_line(self.line1)
        ax1.set_xlim(0, self.x1[-1]+1)
        ax1.set_ylim(min(elev), 240)
        #ax1.set_aspect('equal', 'datalim')

        #ax2.set_xlabel('x')
        ax2.set_ylabel('H(m)')
        self.line2 = Line2D([], [], color='black')
        ax2.add_line(self.line2)
        ax2.set_xlim(0, self.x2[-1]+1)
        ax2.set_ylim(min(elev), 240)

        animation.TimedAnimation.__init__(self, fig, interval=50, blit=True)

    def _draw_frame(self, framedata):
        index = framedata

        H_interest = []    
        H_interest_tfixed = []
        H_interest_combine = []
        for j in pipe_interest1:  
            #this returns H as a function of x in pipe j at time step m
            Hx = n1.pressureSpaceSeries(j,M_start+index*Mi_draw) 
            # Add elevation to the pressure head
            # nd.array can add list to get a new array like matrix sum
            edge = n1.conn[j]
            left = elev[edge[0]]
            right = elev[edge[1]]
            diff = right - left
            Nj = n1.Ns[j]
            He = [left + diff/Nj*(n1+0.5)  for n1 in xrange(Nj)]
            Hx = Hx + He
            H_interest_tfixed.append(Hx)
        H_interest.append(H_interest_tfixed)
        H_interest_combine = np.concatenate(H_interest[index],axis =0)
        self.line1.line.set_data(self.x1, H_interest_combine)
        ax1.set_xlabel('x (m), t=%ds'%((M_start+index*Mi_draw)*dt))    
            
        H_interest = []    
        H_interest_tfixed = []
        H_interest_combine = []
        for j in pipe_interest2:  
            #this returns H as a function of x in pipe j at time step m
            Hx = n1.pressureSpaceSeries(j,M_start+index*Mi_draw) 
            # Add elevation to the pressure head
            # nd.array can add list to get a new array like matrix sum
            edge = n1.conn[j]
            left = elev[edge[0]]
            right = elev[edge[1]]
            diff = right - left
            Nj = n1.Ns[j]
            He = [left + diff/Nj*(n1+0.5)  for n1 in xrange(Nj)]
            Hx = Hx + He
            H_interest_tfixed.append(Hx)
        H_interest.append(H_interest_tfixed)
        H_interest_combine = np.concatenate(H_interest[index],axis =0)
        self.line2.line.set_data(self.x2, H_interest_combine)
        ax2.set_xlabel('x (m), t=%ds'%((M_start+index*Mi_draw)*dt))       
            
            
            
        self._drawn_artists = [self.line1, 
                               self.line2]
            
    def new_frame_seq(self):
        t_delta=0.1              # unit: s
        t_start=0              # unit:s
        t_end= 300               # unit: s
        dt = n1.T/n1.M
        Mi_draw=int(t_delta/dt)   # difne how many steps to skip
        print Mi_draw, dt
        M_start=int(t_start/dt)
        M_total=int(t_end/dt)
        step = int((M_total-M_start)/Mi_draw)
        return iter(range(M_start, M_total, step))

    def _init_draw(self):
        for pipe in pipe_interest:
            # create x axis
            x = np.linspace(pipe_length,pipe_length+n1.Ls[pipe],n1.Ns[pipe])
    
            # create elevation line
            edge = n1.conn[pipe]
            left = elev[edge[0]]
            right = elev[edge[1]]
            diff = right - left
            He = [left + diff/(len(x)-1)*i  for i in xrange(len(x))]
            ax1.plot(x,  He, 'b')
    
            # create pipe full line
            p0 = PyPipe_ps(n1.Ns[pipe], n1.Ds[pipe],n1.Ls[pipe], M, n1.a[0])
            A0 = n1.Ds[pipe]*n1.Ds[pipe]/4*3.14
            H_full = [p0.Eta(A0,True)/9.81/A0]*len(x) 
            H_full = (np.add(H_full, He)).tolist()
            ax1.plot(x, H_full,'--',label= 'pipe %d full'%(pipe))
    
            # If pipe is also outflow pipe
            if pipe in outflow_pipe:
                A_open = p0.AofH(orifice[out], False)
                H_open= [p0.pbar(A_open,False)]*len(x)
                print Orifice[out],A_open, H_open[0] 
                H_open =  (np.add(H_open, He)).tolist()
                ax1.plot(x,H_open,'r--',label = "pipe %d orifice"%(pipe))
                out += 1
            pipe_length += n1.Ls[pipe]
            
        lines = [self.line1, 
                 self.line2]
        for l in lines:
            l.set_data([], [])

ani = SubplotAnimation()
#ani.save('test_sub.mp4')
plt.show()

<h1> Get Excel Data And Plot <h1>

In [ ]:
######## Read Data From Excel File #####################################3
import pandas
t3 = time.time()
df = pandas.read_excel('/home/xin/pipes/indata/Xindata/August2016Export_Excel.xlsx')
t4 = time.time()
print "Reading Excel Data with time = %d s"%(t4-t3)

In [ ]:
#print the column names
#print df.columns
#get the values for a given column
BeforeT_Avg = df['492334 Chnl 01 Avg'].values    
AfterT_Avg = df['492333 Chnl 01 Avg'].values   
Time = df['TimeStamp'].values
print 'Some problems happen with After T data since the min is 2psi with higher elev.'
###############################################################################
#print values[178494]   ##Trial Function, print 178496 row value, so to get nth row value, should use n-2 index
##################################################################################
##### Find rows we need from excel file #############################
start_row = 173190
end_row = 187241
####################################################################################


BeforeT_Avg_23rd = BeforeT_Avg[int(start_row-2):int(end_row-1)]
AfterT_Avg_23rd = AfterT_Avg[int(start_row-2):int(end_row-1)]
Time_23rd = Time[int(start_row-2):int(end_row-1)]

def getIndex(array, target_string):
    indexes = []
    for i in xrange(len(array)):
        if array[i] == target_string:
            indexes.append(i)
    return indexes

c = 0.7028 # Psi_to_Pressure Head

BeforeT_index = getIndex(BeforeT_Avg_23rd, ' - ')
BeforeT_Avg_23rd = np.delete(BeforeT_Avg_23rd,BeforeT_index)  # BeforeT_Avg_23rd type: np.ndarray
BeforeT_Avg_23rd = [i*c for i in BeforeT_Avg_23rd]
AfterT_index = getIndex(AfterT_Avg_23rd, ' - ')
AfterT_Avg_23rd = np.delete(AfterT_Avg_23rd,AfterT_index)  # AfterT_Avg_23rd type: np.ndarray
AfterT_Avg_23rd = [i*c for i in AfterT_Avg_23rd]

import copy
T1 = copy.deepcopy(Time_23rd)
T2 = copy.deepcopy(Time_23rd)
T1 = np.delete(T1,BeforeT_index)
T2 = np.delete(T2,AfterT_index)

#get a data frame with selected columns
#FORMAT = ['Arm_id', 'DSPName', 'Pincode']
#df_selected = df[FORMAT]



################## Define Start Time String and End Time String Based on Simulation ######################
start_time = '8/23/2016 05:03:05'
end_time = '8/23/2016 05:33:05'
##################################################################################################

delta_t = n1.T/n1.M * 1000  # in milliseconds

# 'start' is a string in H:M:S format, so is 'end'. 'delta' is an int with unit: ms
def date_range(start, end, delta):    
    from datetime import datetime, timedelta
    FMT = "%m/%d/%Y %H:%M:%S"
    start = datetime.strptime(start,FMT)
    end = datetime.strptime(end,FMT) 
    diff = timedelta(milliseconds = delta)
    times = []
    time = start
    while time <= end:
        times.append(time)  #.time() method to extract hour, minutes, seconds and microseconds from datetime
        time = time + diff
    return times
t = date_range(start_time, end_time,delta_t)
print len(t)
print len(P_beforeT)



#----------------------------------------------------------------------------
# Define a function to get time average to suppress oscillation

# t is the initial x value that corresponds to P
# P is the pressure that should be averaged over time
# Mi is how many time steps you want to average 
def t_average_time(t, P, Mi):
    n = int(len(P[:-1])/Mi)    # Totally M steps in P{:-1], n=M/Mi, n should be an integer
    x = []          
    P_new = []
    for i in xrange(n):
        P_new.append(np.average(P[i*Mi:(i+1)*Mi-1]))
        x.append(t[int((i+0.5)*Mi)])
    return x, P_new
#--------------------------------------------------------------
'''
# Get Pressure From Numerical Result
P_beforeT = n1.pressureTimeSeries(2,n1.Ns[2]-2)  #One grid before the last one 
P_afterT = n1.pressureTimeSeries(4,1)  
'''


Mi = 1000
print "Now the delta t is", Mi*n1.T/n1.M, 's'

t1n, P_beforeTn = t_average_time(t, P_beforeT, Mi)
t2n, P_afterTn = t_average_time(t, P_afterT, Mi)

# Create Plots
fig = plt.figure(figsize= (12,5))
plt.plot(T1, BeforeT_Avg_23rd,'r--',label='Before T')
plt.plot(T2, AfterT_Avg_23rd,'b--',label='After T')
plt.plot(t1n, P_beforeTn, 'r', label ='Numerical Before T')
plt.plot(t2n, P_afterTn,'b', label ='Numerical After T')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Pressure Head (m)')
plt.title('Field Data')

In [ ]:
m32gal = 264.172   #conversion factor, m^3 to gallons
m2psi = 1.42       #conversion factor, m of pressure head to psi
mdx = min([n1.Ls[i]/n1.Ns[i] for i in range(n1.Nedges)])
#print timing information
print "Simulated time T = %f s" %n1.T
print "Solve time = %f  s" %n1.solve_time
print "dt = %f" %dt
print "inflow volume = %.2f gallons" % ((n1.getTotalVolume()-V0)*m32gal)
print "pressure wave speed is %f m/s"% (max(n1.a))
print "CFL = 1/(dx/dt)*(max wave speed) = %f" % (max(n1.cmax)*dt/mdx)

In [ ]:
# i = 0 means A, i=1 means Q
# j means grid number [0,N+1] including two grid cells
# n means time step
# N means grid number of this pipe
def idx_t(i,j,n,N):
    return (2*(N+2)*n+(N+2)*i+j)
H1 = n1.pressureSpaceSeries(0,int(M))  
H2 = n1.pressureSpaceSeries(1,int(M)) 
H3 = n1.pressureSpaceSeries(2,int(M)) 
print len(H1) # so H0 only contains real grid values
print H1[-1],H2[0],H3[0]

# print the pressure head at the junction
#print '0:',p1.pbar(0.151917,True),"1,2:",p1.pbar(0.162652,False)

# get pipe (A,Q)data
qh1 = n1.qhist(0)
qh2 = n1.qhist(1)
qh3 = n1.qhist(2)
N = n1.Ns[1]

# For pipe 1, print Qin,Ain,Qext, Aext (right end)
M = n1.M/2
Ain1 = qh1[idx_t(0,N,int(M),N)]
Aext1 = qh1[idx_t(0,N+1,int(M),N)]
Qin1 = qh1[idx_t(1,N,int(M),N)]
Qext1= qh1[idx_t(1,N+1,int(M),N)]
print "The cell value of pipe 1:",Ain1,Qin1,Aext1,Qext1,p1.Eta(Ain1,False)/9.81/Ain1

# For pipe 2, print Qin,Ain,Qext, Aext (left end)
Ain2 = qh2[idx_t(0,1,int(M),N)]
Aext2 = qh2[idx_t(0,0,int(M),N)]
Qin2 = qh2[idx_t(1,1,int(M),N)]
Qext2= qh2[idx_t(1,0,int(M),N)]
print "The cell value of pipe 2:",Ain2,Qin2,Aext2,Qext2,p1.Eta(Ain2,False)/9.81/Ain1

# For pipe 3, print Qin,Ain,Qext, Aext (left end)
Ain3 = qh3[idx_t(0,1,int(M),N)]
Aext3 = qh3[idx_t(0,0,int(M),N)]
Qin3 = qh3[idx_t(1,1,int(M),N)]
Qext3= qh3[idx_t(1,0,int(M),N)]
print "The cell value of pipe 3:",Ain3,Qin3,Aext3,Qext3,p1.Eta(Ain3,False)/9.81/Ain1

print len(qh1)



In [ ]:
# Calculated Outflow
#****************************************************************************
t_interval = 1                  # unit: s
pipe_interest = range(0,3)      # Pipes you want to know 
#****************************************************************************

def get_gradient(distance,pressure_series):
    gradient = []
    for k in xrange(len(pressure_series)-1):
        gradi = (pressure_series[k]-pressure_series[k+1])/distance
        #if gradi == 0:                     # 0 is a const to reflect initial gradient
            #return 0
        gradient.append(gradi)
    return gradient

Mi_draw = int(t_interval/dt)
average_all=[]
deviation_all=[]
for i in xrange(len(pipe_interest)):
    pipe_number = pipe_interest[i]
    dx = n1.Ls[pipe_number]/n1.Ns[pipe_number]
    t_dev=[]
    deviation = []
    average = []  
    for m in range(0,n1.M+1,Mi_draw):
        Hx = n1.pressureSpaceSeries(pipe_number,m)
        gradient = get_gradient(dx,Hx[2:-2])           # eliminate first two grids and the last grid
        ave = np.mean(gradient)
        gradient = [gradient[loc] / ave for loc in range(len(gradient))]  # normalize gradient
        deviation.append(np.std(gradient))
        average.append(ave)
        t_dev.append(m*dt)
    average_all.append(average)
    deviation_all.append(deviation)
    



# average_all [pipe][time_step]
# Using the average gradient along the pipe to get [This is the equation used in the steady state, so try it]
U_interest=[]
def idx_t(i,j,n,N):
    return (2*(N+2)*n+(N+2)*i+j)

g = 9.81
pipe_interest = range(0,3)
plt.figure(figsize= (12,5)) 
for pipe in pipe_interest:
    mean_gradient = average_all[pipe]
    Q_pipefixed = []
    for time in xrange(len(mean_gradient)):
        #using the equation of pressurized short pipe, assume that the pipe is full
        # Q = miu*A*sqrt(2*g*(H2-H1))
        lambuda = 0.037
        miu = 1/sqrt(1+lambuda*n1.Ls[pipe]/n1.Ds[pipe]) # no obvious local loss, only frictional loss
        delta_head = mean_gradient[time]*(n1.Ns[pipe]-3)/n1.Ns[pipe]*n1.Ls[pipe]
        A = n1.Ds[pipe]**2/4*pi
        if delta_head<0:
            Q=-miu*A*sqrt(abs(2*g*delta_head))
        else:
            Q = miu*A*sqrt(abs(2*g*delta_head))
        Q_pipefixed.append(Q) 
    plt.plot(t_dev,Q_pipefixed,label='pipe%d'%pipe) 
    legend()
    plt.xlabel('t(s)')
    plt.ylabel('Q (m3/s)')
    #plt.title('v=%.1f,Pipe %s Outflow (from me)'%(v,str(pipe_interest)))
    #savefig('/home/xin/pipes/examples/output_data/EpipeFig/Outflow/Pipe %s Outflow (from me).png'%str(pipe_interest), bbox_inches='tight')

    
# Compare with Lieb's results from equation [just get Q_end from each pipe]
Q=[]
for i in range(len(pipe_interest)):
    Q_pipefix = [] 
    Q_end=[]
    for m in xrange(0,n1.M,Mi_draw) :
        j = pipe_interest[i]
        N = n1.Ns[j]
        qh = n1.qhist(j)
        Q_end.append(qh[idx_t(1,N-1,m,N)])     # QN
    Q_pipefix.append(Q_end)
    Q.append(Q_pipefix)
xt = np.linspace(0, n1.M*dt, n1.M/Mi_draw)
plt.figure(figsize= (12,5))             # plot on different figures
for i in range(len(pipe_interest)):   
        print len(xt),len(Q[i][0])
        plot(xt,Q[i][0], lw = 1,label = 'pipe %d'%(pipe_interest[i]))
        #legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)   #legend will be placed out of box and cannot be shown
        legend()
        xlabel('time (s)')
        ylabel('Q (m3/s)')
        #plt.title('v=%.1f,Pipe %s Outflow (from me)'%(v,str(pipe_interest)))
#savefig('/home/xin/pipes/examples/output_data/EpipeFig/Outflow/Pipe %s Outflow (from Libe).png'%str(pipe_interest), bbox_inches='tight')

In [ ]:
# Get inflow to the system and outflow from the system
#****************************************************************************
t_interval = 3.5*dt                  # unit: s
pipe_interest = range(0,3)      # Pipes you want to know 
#****************************************************************************

def get_gradient(distance,pressure_series):
    gradient = []
    for k in xrange(len(pressure_series)-1):
        gradi = (pressure_series[k]-pressure_series[k+1])/distance
        #if gradi == 0:                     # 0 is a const to reflect initial gradient
            #return 0
        gradient.append(gradi)
    return gradient

Mi_draw = int(t_interval/dt)

Q=[]
for i in range(len(pipe_interest)):
    Q_pipefix = [] 
    Q_end=[]
    for m in xrange(0,n1.M,Mi_draw) :
        j = pipe_interest[i]
        N = n1.Ns[j]
        qh = n1.qhist(j)
        if j == 0:
            Q_end.append(qh[idx_t(1,0,m,N)])
        else:
            Q_end.append(qh[idx_t(1,N-1,m,N)])     # QN
    Q_pipefix.append(Q_end)
    Q.append(Q_pipefix)
xt = np.linspace(0, n1.M*dt, n1.M/Mi_draw+1)
plt.figure(figsize= (12,5))             # plot on different figures
for i in range(len(pipe_interest)): 
        print len(xt),len(Q[i][0])
        plot(xt,Q[i][0], lw = 1,label = 'pipe %d'%(pipe_interest[i]))
        legend(bbox_to_anchor=(0.9, 1), loc=2, borderaxespad=0.)   #legend will be placed out of box and cannot be shown
        #legend()
        plt.ylim(0, 0.3)
        xlabel('time (s)')
        ylabel('Q (m3/s)')
        #plt.title('v=%.1f,Pipe %s Outflow (from me)'%(v,str(pipe_interest)))
#savefig('/home/xin/pipes/examples/output_data/MyTtest/inflowq=0.25,v=1.27/Pipe %s Outflow (from Libe).png'%str(pipe_interest), bbox_inches='tight')

In [ ]:
# Water Volume changes in pipe with time

#*****************************************************************************
pipe_interest1=range(0,3)
#*****************************************************************************

print "The volume in each pipe is",(n1.Ds[0]**2*pi/4)*n1.Ls[0], "m^3"

# Get A of two ends of each pipe
A = [] # all time, all pipes, all A values in grids
M_total=n1.M+1
Mi_draw=100

for m in xrange(0,M_total,Mi_draw) :
    A_tfixed=[] # certain time, all pipes, all A values in grids
    for i in xrange(len(pipe_interest1)):
        j = pipe_interest1[i]
        N = n1.Ns[j]
        qh = n1.qhist(j) 
        Atemp=[]  #certain time, certain pipe, all A values in grids
        for k in xrange(1,N+1):
            Ak=qh[idx_t(0,k,m,N)]      
            Atemp.append(Ak)
        A_tfixed.append(Atemp)     
    A.append(A_tfixed)
        
#calculate total volumn, i means time, j means pipe number, distance means value of grids  
V_total=[]
for i in xrange(0,M_total,Mi_draw):
    V_total_tfix=[]
    for j in xrange(len(pipe_interest1)):
        V_sum=0
        pipe_number=pipe_interest1[j]
        delta_x=n1.Ls[pipe_number]/n1.Ns[pipe_number]
        for distance in xrange(len(A[int(i/Mi_draw)][j])):
            V_sum += A[int(i/Mi_draw)][j][distance]*delta_x
        V_total_tfix.append(V_sum)
        #print 'Pipe %d during %d s have total flow: %.3f m^3 '%(pipe_number,i*dt,V_sum)
    #print 'The total volumn at t=%d s are                      %.3f m^3'%(i*dt,sum(V_total_tfix))
    V_total.append(V_total_tfix)

xt1 = np.linspace(0, M_total*dt, M_total/Mi_draw)
plt.figure(figsize= (11,5))
title('Water Volume in Pipe ~ time')
xlabel('t(s)')
ylabel('V(m^3)')
for j in xrange(len(pipe_interest1)):
    pipe_number=pipe_interest1[j]
    V_total_pipefix=[]
    for i in xrange(1,M_total,Mi_draw):
        V_total_pipefix.append(V_total[int(i/Mi_draw)][j])
    plot(xt1,V_total_pipefix,label = 'pipe %d'%pipe_number)
    plt.ylim(0,20)
    legend(bbox_to_anchor=(0.9, 1), loc=2, borderaxespad=0.)



In [ ]:
#show time series at a sensor in the middle of each pipe in the network
# import a nice colormap
from matplotlib import cm
import matplotlib.colors as colors
cNorm  = colors.Normalize(vmin=0, vmax=Np+1)
scalarMap = cm.ScalarMappable(norm=cNorm, cmap=cm.get_cmap('hsv') )
fig = figure(figsize= (15,5))
t = linspace(0,T,M+1)
interesting = arange(0,Np) #pipes you are interested in (currently all)
interesting1 = array([0,1,2,4,6])
for i in interesting1: 
    #look at pressure time series in i^th pipe, pbar(A(X*, t)), where X* = L/2 for pipe length L, and t =0, dt, 2*dt,...dt*M
    P1 = n1.pressureTimeSeries(i,n1.Ns[i]/2-1) 
    plot(t,P1,color =scalarMap.to_rgba(i), label = "pipe %d" %i) 
legend(ncol=7, loc = (0,1))
ax = gca()
ax.set_xlabel('t (s)')
#xlabel('t(s)')
#ylabel('H(m)')
ax.set_ylabel('H (m)')

In [ ]:
#show pressure time series at different locations along the length of a single pipe
i = 1  #pipe number you want to look at
fig = figure(figsize= (15,5))
#new color map
xs = np.arange(1,n1.Ns[i]+1,4)           # all the grids, increment is 2
print xs
print len(xs)
cNorm  = colors.Normalize(vmin=0, vmax=xs[-1]+10)
scalarMap = cm.ScalarMappable(norm=cNorm, cmap=cm.get_cmap('ocean') )
t = np.linspace(0,n1.T,M+1)
for J in range(0,len(xs),3):
    cval = scalarMap.to_rgba(xs[J])
    dx = n1.Ls[i]/float(n1.Ns[i])
    P2 = n1.pressureTimeSeries(i,J)
    q=plot(t,P2,color =cval,label="x = %.1f"%(dx*xs[J]))
ax = gca()
ax.set_title('Pipe %d'%i)
ax.set_xlabel('t (s)')
ax.set_ylabel(r'$\bar{p}$ (m)')
#legend()
#legend(loc = 'upper left')
#savefig("../sensors_pipe%d.eps"%i, format='eps')

In [ ]:
#show how pressure profile varies in space at the end of the run 
i = 5  # pipe you want to look at
fig = figure(figsize= (10,5))
#m = M  #time slice you want to look at, the simulation only has M steps, so m here refers to the end of the simulation
for m in range(0,n1.M+1,6000):
    Px = n1.pressureSpaceSeries(i,m)
    x = linspace(0,n1.Ls[i],n1.Ns[i])
    plot(x,Px,label = 't=%.2f s'%(dt*m))
    # title('Pipe %d at time %.2f s'%(i,dt*m))
    legend()
    draw()
    xlabel('x')
    ylabel(r'$\bar{p}(m)$')
    fig = figure(figsize= (10,5))


In [ ]:
#plot network layout
(xs,ys,conns,ls) = getBasicConnectivity(fi)
Np = shape(conns)[0]
plotNetworkLayout(xs,ys,conns,ls,Np)

In [ ]:
# The following codes are only used to try python functions, the purpose is about learning python
plt.plot([1,2,3,4], [1,4,9,16], 'ro')
plt.axis([0, 6, 0, 20])
plt.show()


In [ ]:
help(gca)

In [ ]:

1*np.ones(2)

In [ ]:
# Continuous Pressure Head Display
from __future__ import division
import sys
sys.path.append("..")    
from allthethings import PyNetwork, PyPipe_ps
from allthethings import PyBC_opt_dh
import numpy as np
import matplotlib.pyplot as plt
#%pylab inline
%pylab notebook
from writeit import *
import pickle
import time

#*********************************************************************************
pipe_interest=[2,3,9]
t_delta=1              # unit: s
t_start=0               # unit:s
t_end=3600               # unit: s
#*********************************************************************************

# create x axis
x_interest=[]
pipe_length=0
for j in pipe_interest: 
    x = np.linspace(0,n1.Ls[j],n1.Ns[j])
    x_interest.append(x)

#create initial figure
from matplotlib import animation
fig = plt.figure(figsize= (10,5))
plt.xlim(0,x_interest[-1][-1]+1)
plt.ylim(-0.1, 400)
lines = [plt.plot([], [],label='pipe {}'.format(pipe_interest[i]))[0] for i in range(len(pipe_interest))] # number of lines plot on the figure
plt.xlabel('x (m)')        
plt.ylabel('Pressure Head (m)')
plt.title('Pressure Head in pipe %s'%str(pipe_interest)) 
# initialization function: plot the background of each frame

def init():
    for line in lines:
        line.set_data([], [])
    return lines

Mi_draw=int(t_delta/dt)   # difne how many steps to skip
M_start=int(t_start/dt)
M_total=int(t_end/dt)

def animate(index): 
    for i,line in enumerate(lines):             
        j=pipe_interest[i]
        Hx = n1.pressureSpaceSeries(j,M_start+index*Mi_draw)  
        #this returns H as a function of x in pipe j at time step m
        line.set_data(x_interest[i],Hx)
        plt.xlabel('x (m), t=%ds'%((M_start+index*Mi_draw)*dt))
        #line.set_label('pipe %d t=%.2f s'%(j,(dt*(M_start+index*Mi_draw))))
                      
    return lines         
    #legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    

step = int((M_total-M_start)/Mi_draw)
anim = animation.FuncAnimation(fig, animate, init_func=init,frames=step, interval=10, blit=True)
plt.legend(bbox_to_anchor=(0.95, 1), loc=2, borderaxespad=0.)
plt.show()
#anim.save('/home/xin/pipes/examples/output_data/T_junction_detection/pipe %s %.0f-%.0f s.mp4'%(str(pipe_interest),t_start,t_end))